In [69]:
# Import basic libraries
import pandas as pd
import numpy as np

In [70]:
# Import data. It is a large dataset, so it is split into two csv files
data1 = pd.read_csv('your_data_feb_22_2022_0525pm-part1.csv')
data2 = pd.read_csv("your_data_feb_22_2022_0525pm-part2.csv")

In [71]:
# Combine the two datasets
data = pd.concat([data1, data2], axis=0)

In [72]:
print(data.size)
print(data1.size + data2.size)

145321400
145321400


In [73]:
# Drop redundant columns
data.drop('sg_g__brands', inplace=True, axis=1)
data.drop('sg_g__latitude', inplace=True, axis=1)
data.drop('sg_g__longitude', inplace=True, axis=1)
data.drop('sg_g__street_address', inplace=True, axis=1)
data.drop('sg_g__city', inplace=True, axis=1)
data.drop('sg_g__region', inplace=True, axis=1)
data.drop('sg_g__postal_code', inplace=True, axis=1)
data.drop('sg_g__iso_country_code', inplace=True, axis=1)

In [74]:
data.shape

(2076020, 62)

In [75]:
# Drop the rest of the Polygon Data (This data is not useful because it is redundant with the Places dataset)
data2 = data.drop(["sg_g__location_name","sg_g__polygon_wkt","sg_g__polygon_class","sg_g__includes_parking_lot","sg_g__is_synthetic","sg_g__enclosed","sg_g__wkt_area_sq_meters"], axis=1)

In [76]:
# This shows how many null values are in each column 
data2.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2076020 entries, 0 to 1010739
Data columns (total 55 columns):
 #   Column                                             Non-Null Count    Dtype  
---  ------                                             --------------    -----  
 0   date_range_start                                   2072601 non-null  object 
 1   date_range_end                                     2072601 non-null  object 
 2   placekey                                           2076020 non-null  object 
 3   sg_c__parent_placekey                              245362 non-null   object 
 4   sg_c__location_name                                2067300 non-null  object 
 5   sg_c__safegraph_brand_ids                          499830 non-null   object 
 6   sg_c__brands                                       499830 non-null   object 
 7   sg_c__top_category                                 2061831 non-null  object 
 8   sg_c__sub_category                                 2044394 non

In [138]:
df3 =  data2[data.date_range_start.notna()]
df3 = df3[df3.sg_c__longitude.notna()]
df3 = df3[df3.sg_c__latitude.notna()]

In [139]:
# This subsets out Weekly Patterns dataset because the Places dataset contains discrepancies with location name and street address
df4 = df3.drop(df3.iloc[:, :9].copy(), axis = 1).copy()
df4 = df4.drop(df4.iloc[:, 3:15].copy(), axis = 1).copy()

In [140]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2063881 entries, 0 to 1010739
Data columns (total 34 columns):
 #   Column                                             Dtype  
---  ------                                             -----  
 0   sg_c__naics_code                                   float64
 1   sg_c__latitude                                     float64
 2   sg_c__longitude                                    float64
 3   sg_wp__parent_placekey                             object 
 4   sg_wp__location_name                               object 
 5   sg_wp__street_address                              object 
 6   sg_wp__city                                        object 
 7   sg_wp__region                                      object 
 8   sg_wp__postal_code                                 float64
 9   sg_wp__iso_country_code                            object 
 10  sg_wp__safegraph_brand_ids                         object 
 11  sg_wp__brands                                     

In [141]:
# Clean up the column names
df4.columns = df4.columns.str.replace('sg_wp__',"")

In [142]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2063881 entries, 0 to 1010739
Data columns (total 34 columns):
 #   Column                                      Dtype  
---  ------                                      -----  
 0   sg_c__naics_code                            float64
 1   sg_c__latitude                              float64
 2   sg_c__longitude                             float64
 3   parent_placekey                             object 
 4   location_name                               object 
 5   street_address                              object 
 6   city                                        object 
 7   region                                      object 
 8   postal_code                                 float64
 9   iso_country_code                            object 
 10  safegraph_brand_ids                         object 
 11  brands                                      object 
 12  date_range_start                            object 
 13  date_range_end             

In [143]:
# Convert string to datetime so it can be plotted
df4.date_range_start = pd.to_datetime(df4.date_range_start)
df4.date_range_end = pd.to_datetime(df4.date_range_end)

In [144]:
# Data Exploration - I'm trying to see if this is actually null values
df4.visitor_country_of_origin.describe()

count      2039538
unique        8269
top       {"US":4}
freq        319439
Name: visitor_country_of_origin, dtype: object

In [145]:
# A large amount of the values in this column are {}, so there is missing information. 
df4.visitor_home_cbgs.describe()

count     2063881
unique    1204278
top            {}
freq       433596
Name: visitor_home_cbgs, dtype: object

In [146]:
# The same thing happens in this column, so these two columns are mostly null.
df4.visitor_home_aggregation.describe()

count     2063881
unique    1142690
top            {}
freq       424454
Name: visitor_home_aggregation, dtype: object

In [147]:
df5 = df4.drop(['visitor_home_cbgs', 'visitor_home_aggregation'], axis = 1).copy()

In [148]:
df5.shape

(2063881, 32)

In [149]:
# Drop any values without an NAICS code (business code)
df6 = df5[df5['sg_c__naics_code'].notna()]
df6.shape

(2058429, 32)

In [150]:
# Zipcodes for downtown Charleston
zipcodes = [29401, 29402, 29403, 29405, 29409, 29413, 29424, 29425]

In [151]:
# This subsets the data into just data from downtown Charleston
zip = df6['postal_code'].isin(zipcodes)
df7 = df6[zip]

In [152]:
df7.shape

(335769, 32)

In [153]:
# Subset out MUSC and Roper Hospitals because they are not tourist locations

musc = df7['location_name'] != 'Musc Medical Center'
roper = df7['location_name'] != 'Roper Hospital'
df8 = df7[musc & roper]

In [154]:
# Subset out any values above the interstate (outside of the peninsula)
df9 = df8[df8['sg_c__latitude'] < 32.803]

In [162]:
# We had issues with uploading data with date connected to it into WEKA, so we dropped it from the WEKA upload and removed duplicate data from our set
df9[['raw_visit_counts', 'sg_c__latitude', 'sg_c__longitude']].drop_duplicates().to_csv('weka_upload.csv')